# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 87 new papers today
          39 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/38 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.05066


extracting tarball to tmp_2212.05066...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.05085


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.05066/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'Sections/04_Discussion_Conclusions' from 'tmp_2212.05066/Sections/04_Discussion_Conclusions.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'Sections/03_Results' from 'tmp_2212.05066/Sections/03_Results.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/host

...

/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.05985/natbib.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2212.06051...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.06058


extracting tarball to tmp_2212.06058...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.05135-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.05135) | **Photometric calibration in u-band using blue halo stars**  |
|| <mark>Shuang Liang</mark>, <mark>Anja von der Linden</mark> |
|*Appeared on*| *2022-12-13*|
|*Comments*| *Accepted for publication in MNRAS*|
|**Abstract**| We develop a method to calibrate u-band photometry based on the observed color of blue galactic halo stars. The galactic halo stars belong to an old stellar population of the Milky Way and have relatively low metallicity. The "blue tip" of the halo population -- the main sequence turn-off (MSTO) stars -- is known to have a relatively uniform intrinsic edge u-g color with only slow spatial variation. In SDSS data, the observed variation is correlated with galactic latitude, which we attribute to contamination by higher-metallicity disk stars and fit with an empirical curve. This curve can then be used to calibrate u-band imaging if g-band imaging of matching depth is available. Our approach can be applied to single-field observations at $|b| > 30^\circ$, and removes the need for standard star observations or overlap with calibrated u-band imaging. We include in our method the calibration of g-band data with ATLAS-Refcat2. We test our approach on stars in KiDS DR 4, ATLAS DR 4, and DECam imaging from the NOIRLab Source Catalog (NSC DR2), and compare our calibration with SDSS. For this process, we use synthetic magnitudes to derive the color equations between these datasets, in order to improve zero-point accuracy. We find an improvement for all datasets, reaching a zero-point precision of 0.016 mag for KiDS (compared to the original 0.033 mag), 0.020 mag for ATLAS (originally 0.027 mag), and 0.016 mag for DECam (originally 0.041 mag). Thus, this method alone reaches the goal of 0.02 mag photometric precision in u-band for the Rubin Observatory's Legacy Survey of Space and Time (LSST). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.05482-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.05482) | **The globular cluster system of the nearest Seyfert II galaxy Circinus**  |
|| C. Obasi, et al. -- incl., <mark>M. Hempel</mark> |
|*Appeared on*| *2022-12-13*|
|*Comments*| *15 pages, 12 figures*|
|**Abstract**| Context. The globular cluster (GC) system of Circinus galaxy has not been probed previously partly because of the location of the galaxy at - 3.8$^\circ$ Galactic latitude which suffers severely from interstellar extinction, stellar crowding, and Galactic foreground contamination. However, the deep near-infrared (NIR) photometry by the VISTA Variables in the Via L\'actea Extended Survey (VVVX) in combination with the precise astrometry of Gaia EDR3 allow us to map GCs in this region. Aims. Our long-term goal is to study and characterise the distributions of GCs and Ultra-compact dwarfs of Circinus galaxy which is the nearest Seyfert II galaxy. Here we conduct the first pilot search for GCs in this galaxy. Methods. We use NIR VVVX photometry in combination with Gaia EDR3 astrometric features such as astrometric excess noise and BP/RP excess factor to build the first homogeneous catalogue of GCs in Circinus galaxy. A robust combination of selection criteria allows us to effectively clean interlopers from our sample. Results. We report the detection of$\sim$ 70 GC candidates in this galaxy at a 3 $\sigma$ confidence level. They show a bimodal colour distribution with the blue peak at (G-Ks)$_0$ = 0.985$\pm$0.127 mag with a dispersion of 0.211$\pm$0.091 mag and the red peak at (G-Ks)$_0$ = 1.625$\pm$0.177 mag with a dispersion of 0.482$\pm$0.114 mag. A GC specific frequency (S$_N$) of 1.3$\pm$0.2 was derived for the galaxy, and we estimated a total population of 120$\pm$40 GCs. Based on the projected radial distribution it appears that Circinus has a different distribution of GC candidates than MW and M31. Conclusions. We demonstrate that Circinus galaxy hosts a sizeable number of cluster candidates. This result is the first leap towards understanding the evolution of old stellar clusters in this galaxy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.05868-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.05868) | **J-PLUS DR3: Galaxy-Star-Quasar classification**  |
|| R. von Marttens, et al. -- incl., <mark>C. Hernández-Monteagudo</mark>, <mark>H. Vázquez Ramió</mark> |
|*Appeared on*| *2022-12-13*|
|*Comments*| *14 pages, 17 figures*|
|**Abstract**| The Javalambre Photometric Local Universe Survey (J-PLUS) is a 12-band photometric survey using the 83-cm JAST telescope. Data Release 3 includes 47.4 million sources (29.8 million with $r \le 21$) on 3192 deg$^2$ (2881 deg$^2$ after masking). J-PLUS DR3 only provides star-galaxy classification so that quasars are not identified from the other sources. Given the size of the dataset, machine learning methods could provide a valid alternative classification and a solution to the classification of quasars. Our objective is to classify J-PLUS DR3 sources into galaxies, stars and quasars, outperforming the available classifiers in each class. We use an automated machine learning tool called {\tt TPOT} to find an optimized pipeline to perform the classification. The supervised machine learning algorithms are trained on the crossmatch with SDSS DR12, LAMOST DR7 and \textit{Gaia} DR3. We checked that the training set of about 570 thousand galaxies, one million stars and 220 thousand quasars is both representative and pure to a good degree. We considered 37 features: besides the twelve photometric bands with their errors, six colors, four morphological parameters, galactic extinction with its error and the PSF relative to the corresponding pointing. After exploring numerous pipeline possibilities through the TPOT genetic algorithm, we found that XGBoost provides the best performance: the AUC for galaxies, stars and quasars is above 0.99 and the average precision is above 0.99 for galaxies and stars and 0.94 for quasars. XGBoost outperforms the star-galaxy classifiers already provided in J-PLUS DR3 and also efficiently classifies quasars. We also found that photometry was very important in the classification of quasars, showing the relevance of narrow-band photometry. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.05985-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.05985) | **Interstellar Polarization Survey II: General Interstellar Medium**  |
|| M. J. F. Versteeg, et al. -- incl., <mark>R. Santos-Lima</mark> |
|*Appeared on*| *2022-12-13*|
|*Comments*| *19 pages, 15 figures, accepted for publication in AJ*|
|**Abstract**| Magnetic fields permeate the entire Galaxy and are essential to, for example, the regulation of several stages of the star formation process and cosmic ray transportation. Unraveling its properties, such as intensity and topology, is an observational challenge that requires combining different and complementary techniques. The polarization of starlight due to the absorption by field-aligned non-spherical dust grains provides a unique source of information about the interstellar magnetic field in the optical band. This work introduces a first analysis of a new catalog of optical observations of linearly polarized starlight in the diffuse interstellar medium (ISM), the Interstellar Polarization Survey, General ISM (IPS-GI). We used data from the IPS-GI, focusing on 38 fields sampling lines of sight in the diffuse medium. The fields are about 0.3$^{\circ}$ by 0.3$^{\circ}$ in size and each of them contains $\sim1000$ stars on average. The IPS-GI catalog has polarimetric measurements of over 40000 stars, over 18000 of which have ${P}/\sigma_{P} > 5$. We added distances and other parameters from auxiliary catalogs to over 36000 of these stars. We analyzed parameter distributions and correlations between parameters of a high-quality subsample of 10516 stars (i.e. $\sim275$ stars per field). As expected, the degree of polarization tends to increase with the extinction, producing higher values of polarization at greater distances or at lower absolute Galactic latitudes. Furthermore, we find evidence for a large-scale ordered Galactic magnetic field. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06051-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06051) | **Evidence for Populations-dependent vertical motions and the Long-lived  Non-Steady Lopsided Milky Way Warp**  |
|| <mark>Xiang Li</mark>, et al. |
|*Appeared on*| *2022-12-13*|
|*Comments*| *15 pages, 13 figures. Accepted for publication in The Astrophysical Journal*|
|**Abstract**| We present the Galactic disk vertical velocity analysis using OB type stars (OB), Red Clump stars (RC), and Main-Sequence-Turn-Off stars (MSTO) with different average age populations crossed matched with LAMOST DR5 and Gaia DR3. We reveal the vertical velocities of the three populations varies clearly with the Galactocentric distance ($R$) and the younger stellar population has stronger increasing trend in general. The bending and breathing modes indicated by the vertical motions are dependent on the populations and they are varying with spatial locations. These vertical motions may be due to the Galactic warp, or minor mergers, or non-equilibrium of the disk. Assuming the warp is the dominant component, we find that the warp amplitude ($\gamma$, $Z_\omega$) for OB (younger population) is larger than that for RC (medium population) and the later one is also larger than that for MSTO (older population), which is in agreement with other independent analyses of stellar density distribution, and supports the warp is long-lived, non-steady structure and has time evolution. This conclusion is robust whether or not the line-of-nodes $\phi_w$ is fixed or as a free parameter (with $\phi_w$ is around 3$-$8.5$^{\circ}$ as best fit). Furthermore, we find that warp is lopsided with asymmetries along azimuthal angle ($\phi$). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06058-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06058) | **Striped Jets in Post Neutron Star Merger Systems**  |
|| Emma Kaufman, et al. -- incl., <mark>I.M. Christie</mark> |
|*Appeared on*| *2022-12-13*|
|*Comments*| **|
|**Abstract**| Models invoking magnetic reconnection as the particle acceleration mechanism within relativistic jets often adopt a gradual energy dissipation profile within the jet. However, such a profile has yet to be reproduced in first-principles simulations. Here, we perform a suite of 3D general relativistic magnetohydrodynamic simulations of post-neutron star merger disks with an initially purely toroidal magnetic field. We explore the variations in both the microphysics (e.g., nuclear recombination, neutrino emission) and system parameters (e.g., disk mass). In all our simulations, we find the formation of magnetically striped jets. The stripes result from the reversals in the poloidal magnetic flux polarity generated in the accretion disk. The simulations display large variations in the distributions of stripe duration, $\tau$, and power, $\langle P_{\Phi} \rangle$. We find that more massive disks produce more powerful stripes, the most powerful of which reaches $\langle P_{\Phi} \rangle \sim 10^{49}$~erg~s$^{-1}$ at $\tau \sim 20$~ms. The power and variability that result from the magnetic reconnection of the stripes agree with those inferred in short duration gamma-ray bursts. We find that the dissipation profile of the cumulative energy is roughly a power-law in both radial distance, $z$, and $\tau$, with the slope in the range, $\sim 1.7-3$; more massive disks display larger slopes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.05497-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.05497) | **Target Detection Framework for Lobster Eye X-Ray Telescopes with Machine  Learning Algorithms**  |
|| Peng Jia, <mark>Wenbo Liu</mark>, <mark>Yuan Liu</mark>, Haiwu Pan |
|*Appeared on*| *2022-12-13*|
|*Comments*| *Accepted by the APJS Journal. Full source code could be downloaded from the China VO with DOI of this https URL Docker version of the code could be obtained under request to the corresponding author*|
|**Abstract**| Lobster eye telescopes are ideal monitors to detect X-ray transients, because they could observe celestial objects over a wide field of view in X-ray band. However, images obtained by lobster eye telescopes are modified by their unique point spread functions, making it hard to design a high efficiency target detection algorithm. In this paper, we integrate several machine learning algorithms to build a target detection framework for data obtained by lobster eye telescopes. Our framework would firstly generate two 2D images with different pixel scales according to positions of photons on the detector. Then an algorithm based on morphological operations and two neural networks would be used to detect candidates of celestial objects with different flux from these 2D images. At last, a random forest algorithm will be used to pick up final detection results from candidates obtained by previous steps. Tested with simulated data of the Wide-field X-ray Telescope onboard the Einstein Probe, our detection framework could achieve over 94% purity and over 90% completeness for targets with flux more than 3 mCrab (9.6 * 10-11 erg/cm2/s) and more than 94% purity and moderate completeness for targets with lower flux at acceptable time cost. The framework proposed in this paper could be used as references for data processing methods developed for other lobster eye X-ray telescopes. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.05604-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.05604) | **For optical flickering in symbiotic star MWC 560**  |
|| Dragomir Marchev, et al. -- incl., <mark>Nataliya Pavlova</mark> |
|*Appeared on*| *2022-12-13*|
|*Comments*| **|
|**Abstract**| This study is based on observations of MWC560 during the last two observational seasons (2020/2021 and 2021/2022). Other than looking for flickering we were interested in following the variability of brightness in the same period. Looking for similarities in the spectra with other types of stars is also of great interest to us because it could help clarify the stellar configuration of such objects. Our observations during the last two observational seasons of MWC560 confirm the absence of flickering. From the similarities of the gathered spectra of XX Oph and MWC560 we assume that the components in XX Oph are a red giant and a white dwarf, which are also surrounded by a common shell. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

131  publications in the last 7 days.
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers